In [1]:
import numpy as np
import torch,  os
import pandas as pd
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F
import cv2

In [3]:
# load data

train_df = pd.read_csv(r'./train.csv')
test_df = pd.read_csv(r'./test.csv')
train_picture_path=  r'./train'
test_picture_path = r'./test_images/'
zoom=32
IMG_SIZE = zoom

FileNotFoundError: File b'../aoi/train.csv' does not exist

In [ ]:
train_df.head()

In [ ]:
def load_images(x,zoom , dir_path):
    return cv2.resize(cv2.imread(os.path.join(dir_path , x)),(zoom,zoom))

In [ ]:
# Create a Dataset to hold the input data and make it available to dataloader
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X.iloc[i]
        data = np.array(data).astype(np.uint8).reshape(IMG_SIZE, IMG_SIZE, 3)
        
        if self.transforms:
            data = self.transforms(data)
            
            # Also return label if we have it
            if self.y is not None:
                return (data, self.y[i])
            else:
                return data    

In [ ]:
train_df.head()

In [ ]:
train_df['Data'] = train_df['ID'].apply(lambda x : load_images(x,zoom , train_picture_path))
test_df['Data'] = test_df['ID'].apply(lambda x : load_images(x , zoom, test_picture_path))

In [ ]:
all_train_labels=train_df['Label']
all_train_images = train_df['Data']

test_images = test_df['Data']

# Split training data so we have a small validation set and training set
train_images, val_images, train_labels, val_labels = train_test_split(all_train_images, all_train_labels, test_size=0.2)

# Reindex so we can refer to first element using index 0
train_images.reset_index(drop=True, inplace=True)
val_images.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

In [ ]:
# Training Image transformations
train_trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
]
)
# Validation Image transformations
val_trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
]
)


In [ ]:
# Create data loaders
batch_size = 64

train_dataset = MyDataset(train_images, train_labels, train_trans)
val_dataset = MyDataset(val_images, val_labels, val_trans)
test_dataset = MyDataset(test_images, None, val_trans)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# build CNN Model from pytorch tool
class CNN(nn.Module):
    # calculate same padding:
    # (w - k + 2*p)/s + 1 = o
    # => p = (s(o-1) - w + k)/2
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=3,
        stride=1,
        padding=2,
        
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=(2,2),padding=0),
        
        )
        self.conv2 = nn.Sequential(
        nn.Conv2d(16,32,3,1,2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=(2,2),padding=0))
        self.out = nn.Linear(32*8*8,10)
        # optionally initialize weights from Gaussian;
        # Guassian weight init is not recommended and only for demonstration purposes
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                m.weight.data.normal_(0.0, 0.01)
                m.bias.data.zero_()
                if m.bias is not None:
                    m.bias.detach().zero_()
                    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        output = self.out(x)
        
        return output

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
# 兩層捲機 深度3->32
# conv1 --> in(3*32*32) out(16*16*16)
# conv2 --> in(16*16*16) out(32*8*8)
# full-connect layer in = in(32*8*8) out(10)
CNN()

In [ ]:
# Train the model

epochs = 10

train_losses, val_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # Clear last image's results
        optimizer.zero_grad()
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        val_loss = 0
        accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                log_ps = model(images)
                val_loss += criterion(log_ps, labels)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        train_losses.append(running_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        
        print("Epoch: {}/{}.. ".format(e+1, epochs),
             "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
             "Validation Loss: {:.3f}.. ".format(val_loss/len(val_loader)),
             "Accuracy: {:.3f}".format(accuracy/len(val_loader)))
        

In [ ]:
# Plot the losses
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend(frameon=False);